In [1]:
import os
from PIL import Image
import numpy as np
import cPickle as pk

In [2]:
# Function to recursively read images from location
def read_images(location, imageType = ("jpg")):
    
    imageFolders = []
    imageFiles = []
    try:
        fileList = os.listdir(location)
    except OSError:
        return imageFiles, imageFolders
        
    for fl in fileList:
        
        if fl.split(".")[-1] in imageType:
            imageFiles.append(location + "/" + fl)
            imageFolders.append(location.split("/")[-1])
        else:
            subFolder = location + "/" + fl
            folderSub, imageSub = read_images(subFolder)
#             imageFiles.append(imageSub)
#             imageFolders.append(folderSub)
            imageFiles = imageFiles + imageSub
            imageFolders = imageFolders + folderSub
    
    return imageFiles, imageFolders

In [3]:
LOCATION = 'datasets/faces94/'

In [4]:
iFiles, iFolders = read_images(LOCATION)

In [31]:
iFolders = np.array(iFolders)

In [5]:
allImages = np.empty([len(iFiles), 200, 180, 3], dtype="float32")

In [6]:
for i in range(len(iFiles)):
    if(i%500 == 0):
        print i
    im = Image.open(iFiles[i])
    allImages[i] = im
    # train_images = np.append(train_images, im)
    # train_labels = np.append(train_labels, iFolders[i])

0
500
1000
1500
2000
2500
3000


In [34]:
train_index = np.random.choice(a = range(len(allImages)), replace=False, size=2850)

In [38]:
train_images = allImages[train_index,:,:]
test_index = np.setdiff1d(range(len(allImages)), train_index)
test_images = allImages[test_index,:,:]

In [40]:
train_labels = iFolders[train_index]
test_labels = iFolders[test_index]

In [42]:
# Conversion to greyscale
train_greyscale = np.mean(train_images, axis = 3)
test_greyscale = np.mean(test_images, axis = 3)

In [43]:
# Compute mean face
mean_face = np.mean(train_greyscale, axis=0)

In [47]:
# Calculate difference from Mean
diff_faces = train_greyscale - mean_face

In [48]:
# Convert to 1D
train_image1d = diff_faces.reshape([len(diff_faces), 200*180])

In [51]:
train_image1d = train_image1d/255.

In [58]:
# Get Eigen Vectors of covariance matrix
train_cov = np.matmul(train_image1d.T, train_image1d)

KeyboardInterrupt: 

In [55]:
train_cov.shape

(2850, 2850)

In [ ]:
eig_image1d = np.linalg.eig(train_image1d)